## osumapper: create osu! map using Tensorflow and Colab

For mappers who don't know how this colaboratory thing works:
- Press Ctrl+Enter in code blocks to run them one by one
- It will ask you to upload .osu file and audio.mp3 after the third block of code
- .osu file needs to have correct timing (you can use [statementreply](https://osu.ppy.sh/users/126198)'s TimingAnlyz tool)
- After uploading them, wait for a few minutes until download pops

Github: https://github.com/kotritrona/osumapper

### Step 1: Installation

First of all, check the Notebook Settings under Edit tab.<br>
Activate GPU to make the training faster.

Then, clone the git repository and install dependencies.

In [ ]:
#%cd /content/
#!git clone https://github.com/kotritrona/osumapper.git
#%cd osumapper/v7.0
#!apt install -y ffmpeg
#!apt install -y nodejs
#!cp requirements_colab.txt requirements.txt
#!cp package_colab.json package.json
#!pip install -r requirements.txt
#!npm install

/content
Cloning into 'osumapper'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 1270 (delta 60), reused 55 (delta 24), pack-reused 1170
Receiving objects: 100% (1270/1270), 83.37 MiB | 33.48 MiB/s, done.
Resolving deltas: 100% (680/680), done.
/content/osumapper/v7.0
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libc-ares2 nodejs-doc
The following NEW packages will be installed:
  libc-ares2 nodejs nodejs-doc
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 5,585 kB of archives.
After this operation, 24.6 MB of additional disk space wi

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Joe's cell
DRIVE_PATH = '/drive/MyDrive/Cal/Colab/DeepMusicVisualizer/rhythm'

In [6]:
# Richard's cell
DRIVE_PATH = 'drive/MyDrive/cs194/DeeperMusicVisualizer/rhythm'

In [3]:
%cd $DRIVE_PATH

/content/drive/MyDrive/cs194/DeeperMusicVisualizer/rhythm


In [4]:
!apt install -y ffmpeg
!apt install -y nodejs
#!cp requirements_colab.txt requirements.txt
#!cp package_colab.json package.json
!pip install -r requirements_colab.txt
!npm install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libc-ares2 nodejs-doc
The following NEW packages will be installed:
  libc-ares2 nodejs nodejs-doc
0 upgraded, 3 newly installed, 0 to remove and 31 not upgraded.
Need to get 5,585 kB of archives.
After this operation, 24.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 nodejs-doc all 8.10.0~dfsg-2ubuntu0.4 [752 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libc-ares2 amd64 1.14.0-1 [37.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 nodejs amd64 8.10.0~dfsg-2ubuntu0.4 [4,796 kB]
Fetched 5,585 kB in 4s (1,563 kB/s)
Sel

### Step 2: Choose a pre-trained model
Set the select_model variable to one of:
- "default": default model
- "sota": model trained with Sota Fujimori music (☆>5.0)
- "vtuber": model trained with VTuber music (☆4.0-5.3)
- "inst": model trained with rhythm game instrumental music (☆3.5-6.5)
- "tvsize": model trained with TV Size maps (☆3.5-5.0 BPM140-190)
- "hard": model trained with Hard difficulties (☆<3.5 BPM140-190)
- "normal": model trained with Normal difficulties (☆<2.7 BPM140-190)
- "lowbpm": model trained with low BPM music (☆3-4.5 BPM<140)
- "taiko": taiko mode model, experimental (☆3-6)
- "catch": catch mode model, experimental (☆3-6)

In [7]:
from setup_colab import *

select_model = "default"

model_params = load_pretrained_model(select_model);

### Step 3: Upload map and music file<br>
Map file = .osu file with correct timing<br>
Music file = the mp3 file in the osu folder


In [ ]:
from google.colab import files
print("Please upload the map file:")
mapfile_upload = files.upload()
for fn in mapfile_upload.keys():
  uploaded_osu_name = fn
  print('Uploaded map file: "{name}" {length} bytes'.format(name=fn, length=len(mapfile_upload[fn])))
print("Please upload the music file:")
music_upload = files.upload()
for fn in music_upload.keys():
  print('Uploaded music file: "{name}" {length} bytes'.format(name=fn, length=len(music_upload[fn])))


Please upload the map file:


### Step 4: Read the map and convert to python readable format



In [ ]:
from act_newmap_prep import *

step4_read_new_map(uploaded_osu_name);

### Step 5: Use model to calculate map rhythm

Parameters:

"note_density": how many notes will be placed on the timeline, range (0, 1).<br>
"slider_favor": how the model favors sliders against circles, range (-1.1, 1.1).<br>
"dist_multiplier": the distance snap. range (0, +∞). Of course 0/+∞ are not advisable.<br>
"divisor_favor": how the model favors notes to be on each divisor starting from a beat (white, blue, red, blue), range (-1, 1) each.<br>
"slider_max_ticks": the max amount of time a slider can slide, range (1, +∞).

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from act_rhythm_calc import * 

#model = step5_load_model(model_file=model_params["rhythm_model"]);
#npz = step5_load_npz();
#params = model_params["rhythm_param"]

#Run our trained model
model = step5_load_model();
npz = step5_load_npz();
params = step5_set_params(dist_multiplier=1, note_density=0.35, slider_favor=0, divisor_favor=[0] * 4, slider_max_ticks=8);

# Or set the parameters here...
# params = step5_set_params(dist_multiplier=1, note_density=0.32, slider_favor=0, divisor_favor=[0] * 4, slider_max_ticks=8);

predictions = step5_predict_notes(model, npz, params);
converted = step5_convert_sliders(predictions, params);
step5_save_predictions(converted);

(284, 16, 5)
s (4544, 5) [-0.8279239  -0.96900207 -0.94903827 -0.99655944 -0.8591668 ]
[0.43288016]
(4544, 1)
others (4544, 1) [0.]
another (4544, 5)
anoter [0. 0. 0. 0. 0.]
1589 notes predicted.
(4544, 1) (4544, 5) (4547,) (4547,) (4544, 7)
[5430 5513 5597 5680 5763]
[0 1 2 3 4]
1023
(1171,) (1171, 5) (1171,) (1171,) (1171,) (1171,) (1171,) (1171,) (1171,)
slider preds [1. 1. 0. 0. 1.]
slider ticks [2 2 0 0 0]
timestamps [5430 5763 6097 6263 6430]
ticks [ 0  4  8 10 12]
1 8 6097 0.0 0.0 1.0


In [ ]:
print(converted[2][:5])
print(converted[3][:5])
print(converted[-2][:5])
print(converted[4][:5])

[ 0  2  4  8 10]
[5430 5597 5763 6097 6263]
[0 0 2 0 0]
[0. 0. 1. 0. 0.]


## Step 6: Map flow generator

Generate the final map using a Generative Adversarial Network (GAN).

Parameters:


- note_distance_basis: the baseline for distance snap between notes
- max_ticks_for_ds: max number of time ticks (each 1/4) that it uses the distance snap
- next_from_slider_end: use slider end instead of slider head for calculating distance
- box_loss_border, box_loss_value: it's like a barrier on the map edges that bounces off the circles
- divisor, note_group_size: don't change unless you're using a special model built for it
- good_epoch, max_epoch: controls the training time. less time makes it faster but risks less quality
- g_\*, c_\*: hyperparameters used by GAN. No one knows how they work but they mysterically affect the result

In [ ]:
from act_gan import *;

GAN_PARAMS = model_params["gan"]
# Or manually set the parameters...
# GAN_PARAMS = {
#     "divisor" : 4,
#     "good_epoch" : 12,
#     "max_epoch" : 30,
#     "note_group_size" : 10,
#     "g_epochs" : 1,
#     "c_epochs" : 1,
#     "g_batch" : 50,
#     "g_input_size" : 50,
#     "c_true_batch" : 140,
#     "c_false_batch" : 5,
#     "c_randfalse_batch" : 5,
#     "note_distance_basis" : 200,
#     "next_from_slider_end" : False,
#     "max_ticks_for_ds" : 1,
#     "box_loss_border" : 0.1,
#     "box_loss_value" : 0.4,
#     "box_loss_weight" : 1
# };

step6_set_gan_params(GAN_PARAMS);
osu_a, data = step6_run_all(flow_dataset_npz=model_params["flow_dataset"]);

### Since the generation will take a while...

we can appreciate a nice picture of Cute Sophie!!

<img src="https://i.imgur.com/Ko2wogO.jpg" />



Do a little modding to the map.

Parameters:

- stream_regularizer: fix bad streams. integer for modes (0,1,2,3,4) 0=inactive
- slider_mirror: mirror slider ends if they go outside map area. (0,1) 0=inactive 1=active

In [ ]:
from act_modding import *

modding_params = model_params["modding"]
# modding_params = {
#     "stream_regularizer" : 1,
#     "slider_mirror" : 1
# }

osu_a, data = step7_modding(osu_a, data, modding_params);

Finally, save the data into an .osu file!

In [ ]:
from google.colab import files
from act_final import *
from act_taiko_hitsounds import *

if select_model == "taiko":
    taiko_hitsounds_params = step8_taiko_hitsounds_set_params(divisor=4, metronome_count=4)
    hitsounds = step8_apply_taiko_hitsounds(osu_a, data, hs_dataset=model_params["hs_dataset"], params=taiko_hitsounds_params)
    saved_osu_name = step8_save_osu_file(osu_a, data, hitsounds=hitsounds)
else:
    saved_osu_name = step8_save_osu_file(osu_a, data);

files.download(saved_osu_name)

In [ ]:
# clean up if you want to make another map!
# colab_clean_up(uploaded_osu_name)

That's it! Now you can try out the AI-created map in osu!.

For bug reports and feedbacks either report it on github or use discord: <br>
[https://discord.com/invite/npmSy7K](https://discord.com/invite/npmSy7K)